# Eye-Tracking Data Analysis for PLR Tests
This notebook implements analysis of Pupillary Light Reflex (PLR) tests using eye-tracking data.
The analysis includes data cleaning, pupil size extraction, signal quality assessment, and biomarker calculation.

In [ ]:
# Import required libraries

# pandas - For data manipulation and analysis of tabular data (landmarks and protocol files)
import pandas as pd

# numpy - For numerical operations and array manipulation of pupil measurements
import numpy as np

# matplotlib - For creating static visualizations of pupil response data
import matplotlib.pyplot as plt

# pathlib - For cross-platform file path handling
from pathlib import Path

# logging - For tracking program execution and debugging information
import logging

# scipy.signal - For signal processing operations like Savitzky-Golay filtering
from scipy import signal

# seaborn - For enhanced statistical data visualization built on matplotlib
import seaborn as sns

# shapely.geometry - For polygon area calculation
from shapely.geometry import Polygon

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

## Data Validating Functions
Functions to validate landmarks and protocol files

In [ ]:
def load_landmarks_file(file_path):
    """
    Load and validate landmarks data file
    
    Args:
        file_path (str): Path to landmarks CSV file
        
    Returns:
        pd.DataFrame: Processed landmarks data
    """
    try:
        df = pd.read_csv(file_path)
        
        # Required columns for timestamp and status
        base_columns = ['timestamp', 'id', 'retcode']
        
        # Generate expected landmark column names for both eyes
        landmark_columns = []
        for eye in ['left', 'right']:
            for i in range(1, 28):  # 27 landmarks per eye
                landmark_columns.extend([
                    f'{eye}_lm_{i}_x',
                    f'{eye}_lm_{i}_y'
                ])
        
        required_columns = base_columns + landmark_columns
        
        # Verify all required columns exist
        missing_columns = [col for col in required_columns if col not in df.columns]
        if missing_columns:
            raise ValueError(f"Missing required columns: {', '.join(missing_columns)}")
            
        logger.info(f"Successfully validated landmarks file {file_path.name} with {len(df)} rows")
        return df
    except Exception as e:
        logger.error(f"Error loading landmarks file: {e}")
        raise

In [ ]:
def load_protocol_file(file_path):
    """
    Load and validate protocol data file
    
    Args:
        file_path (str): Path to protocol CSV file
        
    Returns:
        pd.DataFrame: Protocol timing data
    """
    try:
        df = pd.read_csv(file_path)
        required_columns = ['time', 'event']
        if not all(col in df.columns for col in required_columns):
            raise ValueError("Missing required columns in protocol file")
        return df
    except Exception as e:
        logger.error(f"Error loading protocol file: {e}")
        raise

In [ ]:
data_path = Path().cwd().parent / "data"
for subfolder in data_path.iterdir():
    if subfolder.is_dir():
        landmarks_file = subfolder / f'{subfolder.name}_plr_landmarks.csv'
        protocol_file = subfolder / f'{subfolder.name}_plr_protocol.csv'
        
        # Load landmarks and protocol files
        landmarks_df = load_landmarks_file(landmarks_file)
        protocol_df = load_protocol_file(protocol_file)

## Data Cleaning
Functions to clean and preprocess the landmarks data

In [ ]:
def clean_landmarks_data(df):
    """
    Clean landmarks data by removing invalid frames
    
    Args:
        df (pd.DataFrame): Raw landmarks data
        
    Returns:
        pd.DataFrame: Cleaned landmarks data
    """
    # Remove frames with invalid retCode
    cleaned_df = df[df['retcode'] == 'OK'].copy()
    
    # Reset index after filtering
    cleaned_df.reset_index(drop=True, inplace=True)
    
    logger.info(f"Removed {len(df) - len(cleaned_df)} frames with invalid retCode")
    return cleaned_df

In [ ]:
# Find the landmarks.csv file
landmarks_file = list(data_path.glob('**/*landmarks.csv'))[0]

# Load landmarks data into DataFrame
landmarks_df = pd.read_csv(landmarks_file)

# Display first few rows
landmarks_df.head()

## Pupil Size Extraction
Functions to calculate pupil size and convert to millimeters

In [ ]:
def visualize_pupil_polygons(landmarks_df, frame_idx=0):
    """
    Visualize pupil landmarks and polygons for both eyes
    
    Args:
        landmarks_df (pd.DataFrame): Landmarks data
        frame_idx (int): Index of the frame to visualize
    """
    # Get single frame data
    frame = landmarks_df.iloc[[frame_idx]]
    
    # Create figure with subplots for left and right eyes
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # Eye names for iteration
    eyes = ['left', 'right']
    
    # Process each eye
    for i, eye in enumerate(eyes):
        ax = axes[i]
        
        # Get center (landmark 8)
        center_x = frame[f'{eye}_lm_8_x'].iloc[0]
        center_y = frame[f'{eye}_lm_8_y'].iloc[0]
        
        # Get regular polygon points
        landmark_sequence = [7, 25, 9, 22, 10, 23, 24, 7]
        regular_points = []
        for lm in landmark_sequence:
            x = frame[f'{eye}_lm_{lm}_x'].iloc[0]
            y = frame[f'{eye}_lm_{lm}_y'].iloc[0]
            regular_points.append((x, y))
        
        # Get boundary points for interpolation
        boundary_points = []
        for lm in [7, 25, 9, 22, 10, 23, 24, 7]:
            x = frame[f'{eye}_lm_{lm}_x'].iloc[0]
            y = frame[f'{eye}_lm_{lm}_y'].iloc[0]
            boundary_points.append((x, y))
        
        # Get interpolated points
        interp_points = []
        # Reuse interpolation code from the existing function
        angles = []
        distances = []
        for p in boundary_points:
            dx = p[0] - center_x
            dy = p[1] - center_y
            angle = np.arctan2(dy, dx)
            distance = np.sqrt(dx**2 + dy**2)
            angles.append(angle)
            distances.append(distance)
            
        angles_sorted = np.sort(angles)
        angles_interp = np.linspace(0, 2*np.pi, 24)
        distances_interp = np.interp(angles_interp, angles_sorted, distances)
        
        for angle, dist in zip(angles_interp, distances_interp):
            x = center_x + dist * np.cos(angle)
            y = center_y + dist * np.sin(angle)
            interp_points.append((x, y))
        
        # Create polygons 
        regular_polygon = Polygon(regular_points)
        interp_polygon = Polygon(interp_points)
        
        # Calculate areas
        regular_area = regular_polygon.area
        interp_area = interp_polygon.area
        
        # Plot regular polygon points
        regular_xs, regular_ys = zip(*regular_points)
        ax.plot(regular_xs, regular_ys, 'bo-', alpha=0.7, markersize=8, linewidth=2, label='Regular Polygon')
        
        # Plot interpolated polygon
        interp_xs, interp_ys = zip(*interp_points)
        ax.plot(interp_xs, interp_ys, 'ro-', alpha=0.5, markersize=5, linewidth=1.5, label='Interpolated Polygon')
        
        # Plot center point
        ax.plot(center_x, center_y, 'go', markersize=10, label='Center (Landmark 8)')
        
        # Plot boundary landmarks with their numbers
        for lm_idx, lm in enumerate([7, 25, 9, 22, 10, 23, 24, 7]):
            x = frame[f'{eye}_lm_{lm}_x'].iloc[0]
            y = frame[f'{eye}_lm_{lm}_y'].iloc[0]
            ax.annotate(str(lm), (x, y), fontsize=12, 
                        xytext=(10, 10), textcoords='offset points',
                        arrowprops=dict(arrowstyle='->', color='black'))
        
        # Set labels and title
        ax.set_title(f'{eye.capitalize()} Eye Pupil\nRegular Area: {regular_area:.2f} px²\nInterpolated Area: {interp_area:.2f} px²', fontsize=14)
        ax.set_xlabel('X Coordinate (pixels)', fontsize=12)
        ax.set_ylabel('Y Coordinate (pixels)', fontsize=12)
        
        # Add legend
        ax.legend(loc='upper left', fontsize=12)
        
        # Invert y-axis as images typically have origin at top left
        ax.invert_yaxis()
        
        # Equal aspect ratio to avoid distortion
        ax.set_aspect('equal')
        
        # Add grid
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Also create a sequence of plots showing the progression of pupil size over time
    if len(landmarks_df) > 5:
        # Create figure for pupil size over time
        plt.figure(figsize=(14, 6))
        
        # Calculate pupil sizes for all frames
        left_areas = []
        right_areas = []
        timestamps = []
        
        for idx in range(min(len(landmarks_df), 30)):  # Limit to first 30 frames for speed
            frame = landmarks_df.iloc[[idx]]
            left_area, right_area = calculate_pupil_size_with_interpolation(frame)
            left_areas.append(left_area)
            right_areas.append(right_area)
            timestamps.append(idx)
        
        # Plot pupil size over time
        plt.plot(timestamps, left_areas, 'b-', linewidth=2, label='Left Eye Pupil Area')
        plt.plot(timestamps, right_areas, 'r-', linewidth=2, label='Right Eye Pupil Area')
        
        plt.title('Pupil Area Over Time', fontsize=14)
        plt.xlabel('Frame Index', fontsize=12)
        plt.ylabel('Pupil Area (square pixels)', fontsize=12)
        plt.legend(fontsize=12)
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Visualize the pupil polygons for the first frame
visualize_pupil_polygons(landmarks_df_cleaned, frame_idx=0)